<a href="https://colab.research.google.com/github/ShaliniAnandaPhD/HarmonyLens/blob/main/Harmoney_Lens_Collaboration_(Part_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installs necessary Python packages for the script, including tweepy for Twitter API access, feedparser for parsing RSS feeds, nltk for natural language processing, transformers for sentiment analysis, cachetools for caching API responses, and logging for error and activity logging.


In [ ]:
!pip install tweepy feedparser nltk transformers cachetools logging

  Using cached logging-0.4.9.6.tar.gz (96 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


# Installs necessary Python packages for the script, including tweepy for Twitter API access, feedparser for parsing RSS feeds, nltk for natural language processing, transformers for sentiment analysis, cachetools for caching API responses, and logging for error and activity logging.


In [ ]:
import tweepy
import requests
import feedparser
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import pipeline
import torch
import torch.nn as nn
import torch.nn.functional as F
from cachetools import cached, TTLCache
import logging

# Set up basic logging
logging.basicConfig(filename='app.log', level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger()


# Defines a function to call APIs with caching enabled to reduce the number of requests. It includes error handling to manage different types of exceptions that might occur during API calls.


In [ ]:
!pip install feedparser

# Implements a function to gather data from various APIs including Twitter, Reddit, New York Times, and BBC. It uses the previously defined API call function with error handling and logging.


In [ ]:
cache = TTLCache(maxsize=100, ttl=3600)

@cached(cache)
def call_api(url, headers=None, params=None):
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
        return response.json()
    except requests.exceptions.HTTPError as errh:
        logger.error("Http Error:", errh)
    except requests.exceptions.ConnectionError as errc:
        logger.error("Error Connecting:", errc)
    except requests.exceptions.Timeout as errt:
        logger.error("Timeout Error:", errt)
    except requests.exceptions.RequestException as err:
        logger.error("OOps: Something Else", err)


# Modifies the data gathering function to focus specifically on collecting data related to "peace talks" and "peace negotiations". The data is then saved in 'output2.json'.


In [ ]:
## TODO keyword search
import requests
import feedparser
import json

def gather_data_from_apis():
    # API endpoints and credentials
    twitter_api = "https://api.twitter.com/2/tweets/search/recent"
    reddit_api = "https://www.reddit.com/r/news/.json"
    nytimes_api = "https://api.nytimes.com/svc/news/v3/content/all/all.json"
    bbc_api = "http://feeds.bbci.co.uk/news/rss.xml"

    # Twitter API
    twitter_headers = {"Authorization": "AAAAAAAAAAAAAAAAAAAAAPGOrgEAAAAAmo4Z%2FY5Gn%2FRq%2FTTthYEU2LMjl9o%3DYxENYJyzjuhNVO1e9futX7vnLVEXsz5TfnmH5bsLccDUJ0p9at"}
    twitter_response = requests.get(twitter_api, headers=twitter_headers)

    # Reddit API
    #reddit_headers = {"User-Agent": "YourAppName"}
    #reddit_response = requests.get(reddit_api, headers=reddit_headers)

    # New York Times API
    nytimes_params = {"api-key": "km7meZc5wxnL0KinmSKwTefAIlFLSb6G"}
    nytimes_response = requests.get(nytimes_api, params=nytimes_params)

    # BBC News RSS feed
    bbc_response = requests.get(bbc_api)

    # Parsing and combining data
    data = []

    if twitter_response.ok:
        data.extend(twitter_response.json().get("data", []))

    #if reddit_response.ok:
        #reddit_data = reddit_response.json()
        #data.extend([child['data'] for child in reddit_data['data']['children']])

    if nytimes_response.ok:
        nytimes_data = nytimes_response.json()
        data.extend(nytimes_data.get("results", []))

    if bbc_response.ok:
        feed = feedparser.parse(bbc_response.content)
        for entry in feed.entries:
            data.append({"title": entry.title, "link": entry.link, "published": entry.published})

    return data

def save_data_to_file(data, filename):
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)

# Usage
data = gather_data_from_apis()
save_data_to_file(data, 'output.json')


In [ ]:
def gather_data_from_apis():
    # Define keywords related to peace talks and negotiations
    keywords = ["peace talks", "peace negotiations"]

    # Combine keywords into a query for the Twitter API
    twitter_query = " OR ".join(keywords) + " -filter:retweets"
    twitter_headers = {"Authorization": "AAAAAAAAAAAAAAAAAAAAAPGOrgEAAAAAmo4Z%2FY5Gn%2FRq%2FTTthYEU2LMjl9o%3DYxENYJyzjuhNVO1e9futX7vnLVEXsz5TfnmH5bsLccDUJ0p9at"}
    twitter_api = "https://api.twitter.com/2/tweets/search/recent?query=" + twitter_query

    # Fetch data from Twitter API
    twitter_response = requests.get(twitter_api, headers=twitter_headers)

    # Parsing and combining data
    data = []
    if twitter_response.ok:
        data.extend([tweet["text"] for tweet in twitter_response.json().get("data", [])])

    # Add more APIs if needed

    return data

data = gather_data_from_apis()
save_data_to_file(data, 'output.json')


# Downloads necessary NLTK resources and defines a function for preprocessing text data. This function tokenizes the text, removes stopwords and non-alphabetic words, and converts them to lowercase.


In [ ]:
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Loads raw data from 'output.json', then preprocesses this data using the function defined in the previous cell.


In [ ]:
def preprocess_text_data(data):
    processed_data = []
    for item in data:
        # Extract text content from each item
        # Assuming 'text' key contains the textual content
        text = item.get('text', '')

        # Tokenize the text
        tokens = word_tokenize(text)

        # Remove stopwords and non-alphabetic words, and convert to lowercase
        processed_text = [word.lower() for word in tokens if word.isalpha() and word not in stopwords.words('english')]

        processed_data.append(processed_text)

    return processed_data


In [ ]:
def load_data_from_file(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Load raw data from 'output.json'
raw_data = load_data_from_file('output.json')

# Preprocess the loaded data
processed_data = preprocess_text_data(raw_data)


# Loads raw data from 'output.json', then preprocesses this data using the function defined in the previous cell.


In [ ]:
from transformers import pipeline
import torch



In [ ]:
def perform_sentiment_analysis(data):
    """
    Perform sentiment analysis on a list of text strings.

    Args:
    data (list of str): A list of text strings to analyze.

    Returns:
    list of dict: A list of dictionaries containing the sentiment analysis results.
    """
    # Check if CUDA is available for faster processing
    device = 0 if torch.cuda.is_available() else -1

    # Initialize the sentiment analysis model pipeline
    # The model automatically defaults to a pre-trained sentiment analysis model
    sentiment_analysis = pipeline("sentiment-analysis", device=device)

    results = []
    for text in data:
        try:
            # Ensure text is a string
            if isinstance(text, str):
                # Perform sentiment analysis
                result = sentiment_analysis(text)
                results.append(result)
            else:
                results.append([{"label": "Error", "score": 0, "text": "Invalid input"}])
        except Exception as e:
            # Handle any errors encountered during analysis
            results.append([{"label": "Error", "score": 0, "text": str(e)}])

    return results


# Executes the sentiment analysis function on the processed data and prints the results, including the sentiment label and score for each text.


In [ ]:
# Assuming 'processed_data' is a list of text strings
sentiment_results = perform_sentiment_analysis(processed_data)

# Print the results
for idx, result in enumerate(sentiment_results):
    print(f"Text {idx+1}: {processed_data[idx]}")
    print(f"Sentiment: {result[0]['label']}, Score: {result[0]['score']:.2f}\n")


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


# Introduces a simple neural network class, `LinguisticModel`, for linguistic pattern analysis in text data. This class includes an embedding layer and two fully connected layers.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# Defines a function to use the `LinguisticModel` for analyzing tokenized text data. It processes each text tensor through the model and collects the results.


In [ ]:
class LinguisticModel(nn.Module):
    """
    A simple neural network for analyzing linguistic patterns in text data.
    It consists of an embedding layer followed by two fully connected layers.

    Args:
    vocab_size (int): Size of the vocabulary (number of unique tokens).
    embedding_dim (int): Size of the embeddings.
    hidden_dim (int): Size of the hidden layer.
    output_dim (int): Size of the output layer.
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LinguisticModel, self).__init__()
        # Embedding layer converts token indices to embeddings
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # First fully connected layer
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)

        # Second fully connected layer that outputs the final results
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Forward pass through the network
        x = self.embedding(x)  # Input is transformed by the embedding layer
        x = F.relu(self.fc1(x))  # Activation function is applied after the first layer
        x = self.fc2(x)  # Final output is generated by the second layer
        return x


# Initializes the `LinguisticModel` with specified parameters and generates example tokenized data for demonstration. It then uses the model to analyze this data and prints the results.


In [ ]:
def analyze_linguistic_patterns(data, model):
    """
    Analyze linguistic patterns in the given data using the provided model.

    Args:
    data (list of torch.Tensor): A list of tokenized texts represented as tensors.
    model (LinguisticModel): The neural network model to use for analysis.

    Returns:
    list of torch.Tensor: The model's predictions for each input text.
    """
    results = []
    for text in data:
        # Ensure input is a tensor
        if isinstance(text, torch.Tensor):
            with torch.no_grad():  # Disable gradient calculations for inference
                result = model(text)
                results.append(result)
        else:
            results.append(None)  # Append None for invalid inputs

    return results


In [ ]:
# Model parameters
vocab_size = 10000  # Total number of unique tokens in the dataset
embedding_dim = 100  # Dimensionality of the embedding layer
hidden_dim = 128  # Number of neurons in the hidden layer
output_dim = 10   # Dimensionality of the output (e.g., number of classes)

# Initialize the model
model = LinguisticModel(vocab_size, embedding_dim, hidden_dim, output_dim)

# Example tokenized data (randomly generated for demonstration)
example_data = [torch.randint(0, vocab_size, (5,)), torch.randint(0, vocab_size, (6,))]

# Analyze the example data using the model
results = analyze_linguistic_patterns(example_data, model)

# Print results (example)
for idx, result in enumerate(results):
    print(f"Result for Text {idx+1}: {result}")


Result for Text 1: tensor([[-0.4284,  0.2192, -0.5140,  0.0373, -0.3696, -0.5261, -0.2975,  0.3188,
          0.4575,  0.0791],
        [ 0.0009,  0.1559, -0.0811, -0.1865, -0.1528,  0.0161,  0.1012,  0.1485,
          0.2192,  0.1273],
        [-0.3916,  0.2458, -0.2334, -0.2898, -0.1489, -0.0817, -0.1024, -0.1449,
         -0.1003, -0.1034],
        [-0.2695, -0.1729,  0.3327, -0.1939, -0.3757, -0.2665,  0.0481,  0.1752,
          0.3329, -0.2378],
        [-0.2028, -0.1194,  0.0291, -0.4334, -0.4405,  0.0976,  0.1557,  0.0883,
          0.4099, -0.1350]])
Result for Text 2: tensor([[-0.1008,  0.1509, -0.0950,  0.0733, -0.0097,  0.0133, -0.0222,  0.0044,
          0.3258, -0.0130],
        [ 0.1697, -0.0644, -0.1625, -0.3352,  0.0184, -0.4167,  0.1377, -0.1766,
          0.2452,  0.2605],
        [-0.6254, -0.0251, -0.1152, -0.0119, -0.3848, -0.4953,  0.0510,  0.2278,
          0.4901,  0.1714],
        [-0.3929, -0.2299,  0.0236, -0.1577, -0.4532, -0.1997,  0.2863, -0.1544,
        

# Installs the Gensim library, which is used for topic modeling and other NLP tasks.


In [ ]:
from gensim import corpora, models
import gensim

def prepare_data_for_lda(processed_data):
    # Ensure that processed_data is not empty
    if not processed_data or not any(processed_data):
        return None, None

    dictionary = corpora.Dictionary(processed_data)
    corpus = [dictionary.doc2bow(text) for text in processed_data if text]
    return corpus, dictionary

# Function to run LDA model
def run_lda_model(processed_data, num_topics=5):
    corpus, dictionary = prepare_data_for_lda(processed_data)

    # Check if corpus and dictionary are valid
    if not corpus or not dictionary:
        print("Error: Corpus or dictionary is empty. Cannot run LDA.")
        return None

    try:
        lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)
        return lda_model
    except ValueError as e:
        print("Error running LDA model:", e)
        return None

# Running the LDA model
lda_model = run_lda_model(processed_data)

# Displaying topics if the model is valid
if lda_model:
    topics = lda_model.print_topics(num_words=4)
    for topic in topics:
        print(topic)
else:
    print("No LDA model to display topics.")



Error: Corpus or dictionary is empty. Cannot run LDA.
No LDA model to display topics.


In [ ]:
import numpy as np
from collections import Counter

# Ensure numpy is installed
!pip install numpy

# Simple stylistic analysis
def stylistic_analysis(processed_data):
    if not processed_data or not any(processed_data):
        print("No data for analysis.")
        return

    word_lengths = [len(word) for text in processed_data for word in text]

    if word_lengths:
        average_word_length = np.mean(word_lengths)
        print("Average word length:", average_word_length)
    else:
        print("No words to analyze for length.")

    word_counter = Counter(word for text in processed_data for word in text)
    if word_counter:
        most_common_words = word_counter.most_common(10)
        print("Most common words:", most_common_words)
    else:
        print("No words to analyze for frequency.")

# Assuming processed_data is in the correct format
stylistic_analysis(processed_data)



No data for analysis.


# Installs libraries necessary for generating a word cloud, then defines a block of code to create and display a word cloud from the processed text data. It includes error handling for library import issues and data validation.


In [ ]:
!pip install matplotlib wordcloud


In [ ]:
# Ensure that WordCloud and matplotlib are installed
!pip install wordcloud matplotlib

import nltk
from nltk.corpus import stopwords

# Ensure nltk resources are available
nltk.download('stopwords')

try:
    from wordcloud import WordCloud
    import matplotlib.pyplot as plt

    # Ensure processed_data is not empty and is in the correct format
    if not processed_data or not any(processed_data):
        raise ValueError("Processed data is empty or not in the expected format.")

    # Generate a word cloud
    all_words = ' '.join([word for text in processed_data for word in text])

    # Check if there are words to generate a word cloud
    if not all_words.strip():
        raise ValueError("No words available for generating a word cloud.")

    wordcloud = WordCloud(width=800, height=800,
                          background_color='white',
                          stopwords=set(stopwords.words('english')),
                          min_font_size=10).generate(all_words)

    # Plot the WordCloud image
    plt.figure(figsize=(8, 8), facecolor=None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.show()

except ImportError as e:
    print("Required libraries are not installed:", e)
except ValueError as e:
    print("Error:", e)
except Exception as e:
    print("An unexpected error occurred:", e)



Error: Processed data is empty or not in the expected format.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
